In [1]:
from torch.utils.tensorboard import SummaryWriter
import os
import surpport.mySQL as mySQL
import surpport.myfunction as MF
from surpport.Args import Args
from surpport.dataprocess import dataload
import torch.nn.functional as F
import torch
import numpy as np

Using backend: pytorch


## 原始模型训练

In [2]:
from torch.utils.data import Dataset
from dgl.data.utils import load_graphs
from os.path import join
from dgl.dataloading import GraphDataLoader

## v1：和 cross_Dataset 差别不大，因为需要先测试行不行，再去测试各种设计
###    因此，保留原始训练集规模，但测试集只最后一个人，因为之前的实验已经
###    已经证明了无法泛化，所以测试集先只保留一个人【先这么做，但实验阐述
###    不可以这么弄】
# todo：修改 dataset 和 opt 来完成预训练
class PreTrain_Dataset(Dataset):
    def __init__(self, arg) -> None:
        super(Dataset, self).__init__()
        self.arg = arg # type: Args
        self.load(arg)
    
    def load(self, arg):
        f = arg.select_f # type: str
        ori = r'E:\DATABASE\FirstGNN\CrossData'
        path = join(ori, f+'_dgl_graph')
        gdata = torch.load(path)
        self.graphs, self.labels = gdata['graphs'], gdata['labels']

    def __getitem__(self, index):
        return self.graphs[index], self.labels[index]

    def __len__(self):
        return len(self.graphs)

def c_dataload(arg, model, opt, alpha):
    """cross exp dataload

    Args:
        arg (Args): Args need
        model (nn.Moudle): ours model
        opt (torch.opt): torch.opt
        alpha (int): 1~10, train set contain by num of patients

    Returns:
        tuple: arg, tr_dataloader, ts_dataloader
    """
    dataset = PreTrain_Dataset(arg)
    if not arg.new_train:
        MF.continue_tr(arg.dir, model, opt, arg)
    else:
        end_num = alpha * 1875 # 单人一个
        lenth = (alpha+1) * 1875 
        arg.tr_id = list(range(end_num))
        arg.ts_id = list(range(end_num, lenth))
    
    train_dataset = torch.utils.data.Subset(
        dataset=dataset, indices=arg.tr_id)
    test_dataset = torch.utils.data.Subset(
        dataset=dataset, indices=arg.ts_id)
    tr_dataloader = dataloader(
        train_dataset, arg.batch_size, collate=MF.collate, shuffle=True)
    ts_dataloader = dataloader(
        test_dataset, arg.batch_size, collate=MF.collate, shuffle=True)
    return arg, tr_dataloader, ts_dataloader

def dataloader(dataset, batch_size, collate, shuffle):
    gdataloader = GraphDataLoader(
        dataset,
        collate_fn=collate,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=False
        # sampler = sampler
    )
    return gdataloader    

In [3]:
from surpport.nnstructure import Classifier7

def m_pre():
    arg = Args()
    # model = GNN.DGCN()
    model = Classifier7(arg)
    # model = EEGNet2()
    model = model.cuda()
    return arg, model

def recorder_build(arg):
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    return recorder

In [4]:
names = ['kly']
fs = [2, ]
for f in fs:
    arg, model = m_pre()
    arg.d_prepare('Pre_Train', f)
    # ! 每次运行都改
    arg.m_info(m_name='m7', m_task='220822_pre_test', num=1)
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    writer = SummaryWriter(arg.tar_path+'\\Journal')

    # ! 根据模型修改
    opt_arg = {'params': model.parameters(),}
    # opt_arg = {'params': model.parameters(),'lr': 6e-5, 'eps': 1e-8, 'weight_decay': 0.1}
    opt = torch.optim.Adam(**opt_arg)

    arg, tr_dataloader, ts_dataloader = c_dataload(arg, model, opt, 9)

In [ ]:
st = 41
step = 40
arg.display_freq = 50
for epoch in range(st, step+st):
    print(epoch)
    recorder[str(epoch)+'-th'] = dict()
    rd = recorder[str(epoch)+'-th']
    # 由于图经过拼合，所以需要多一个dataloader的过程
    # 前两个是list
    tr_args = [epoch, model, opt, tr_dataloader, arg, writer]
    tr_loss, tr_acc = MF.train(*tr_args)
    mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

    tr_args[3] = ts_dataloader
    el_loss, el_acc, logits, labels = MF.evaluate(*tr_args)

    mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
    mySQL.rcd_result(logits, labels, rd)

    val_acc = np.mean(el_acc, axis=0)
    MF.save_best(val_acc, model, arg)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

mySQL.save_recorder(recorder, arg, 'flow')
print('best acc: %.4f' % (arg.best_acc))

In [7]:
ta = np.mean(tr_acc, axis=0)
ta

0.91401404

In [8]:
MF.save_best(ta, model, arg)

已保存模型，best_acc = 0.9140


In [9]:
writer.close()

## 预训练

### 数据准备

In [3]:
from surpport.nnstructure import Classifier7
from surpport.dataprocess import dataload

def m_pre():
    arg = Args()
    # model = GNN.DGCN()
    model = Classifier7(arg)
    # model = EEGNet2()
    model = model.cuda()
    return arg, model

def recorder_build(arg):
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    return recorder

names = ['kly']
fs = [2, ]
for f in fs:
    arg, model = m_pre()
    arg.d_prepare('yc', f)
    # ! 每次运行都改
    arg.m_info(m_name='m7', m_task='220822_pre_test', num=4)
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    writer = SummaryWriter(arg.tar_path+'\\Journal')
    opt = None
    # ! 根据模型修改
    arg, tr_dataloader, ts_dataloader = dataload(arg, model, opt)

### 参数冻结

In [18]:
model.named_parameters

<bound method Module.named_parameters of Classifier7(
  (ConvList): ModuleList(
    (0): Conv1d(128, 96, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(128, 96, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (2): Conv1d(128, 96, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  )
  (GcnList): ModuleList(
    (0): GraphConv(in=96, out=54, normalization=none, activation=None)
    (1): GraphConv(in=96, out=54, normalization=none, activation=None)
    (2): GraphConv(in=96, out=54, normalization=none, activation=None)
  )
  (pressConv): Conv1d(3, 1, kernel_size=(3,), stride=(1,), padding=(1,))
  (Jk_GIN): ModuleList(
    (0): GraphConv(in=54, out=34, normalization=none, activation=None)
    (1): GraphConv(in=34, out=25, normalization=none, activation=None)
    (2): GraphConv(in=25, out=16, normalization=none, activation=None)
  )
  (classify1): Linear(in_features=75, out_features=120, bias=True)
  (classify2): Linear(in_features=120, out_features=5

In [19]:
'classify1' in 'classify1.0'

True

In [22]:
fronze = ['classify1', 'Jk_GIN',
'classify2']
for k, v in model.named_parameters():
    flag = False
    for condi in fronze:
        if condi in k:
            flag = True
            print(k)
            break
    v.requires_grad = flag

Jk_GIN.0.weight
Jk_GIN.0.bias
Jk_GIN.1.weight
Jk_GIN.1.bias
Jk_GIN.2.weight
Jk_GIN.2.bias
classify1.weight
classify1.bias
classify2.weight
classify2.bias


In [23]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

Jk_GIN.0.weight
Jk_GIN.0.bias
Jk_GIN.1.weight
Jk_GIN.1.bias
Jk_GIN.2.weight
Jk_GIN.2.bias
classify1.weight
classify1.bias
classify2.weight
classify2.bias


In [24]:
opt_arg = {'params': filter(lambda p: p.requires_grad, model.parameters()),}
# opt_arg = {'params': model.parameters(),'lr': 6e-5, 'eps': 1e-8, 'weight_decay': 0.1}
opt = torch.optim.Adam(**opt_arg)

### 模型加载

In [25]:
checkpoint = torch.load(r'E:\DATABASE\FirstGNN\220822_pre_test\Pre_Train\m7_1\bmodel.pth')
model.load_state_dict(checkpoint)
# opt.load_state_dict(checkpoint['optimizer'])

<All keys matched successfully>

### 预训练开始

In [ ]:
st = 81
step = 40
arg.display_freq = 5
for epoch in range(st, step+st):
    print(epoch)
    recorder[str(epoch)+'-th'] = dict()
    rd = recorder[str(epoch)+'-th']
    # 由于图经过拼合，所以需要多一个dataloader的过程
    # 前两个是list
    tr_args = [epoch, model, opt, tr_dataloader, arg, writer]
    tr_loss, tr_acc = MF.train(*tr_args)
    mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

    tr_args[3] = ts_dataloader
    el_loss, el_acc, logits, labels = MF.evaluate(*tr_args)

    mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
    mySQL.rcd_result(logits, labels, rd)

    val_acc = np.mean(el_acc, axis=0)
    MF.save_best(val_acc, model, arg)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

mySQL.save_recorder(recorder, arg, 'flow')
print('best acc: %.4f' % (arg.best_acc))

In [29]:
writer.close()